In [24]:
# Import dependencies
import pandas as pd
import numpy as np

import tensorflow
tensorflow.keras.__version__

'2.9.0'

In [25]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

# Import Dataset

In [26]:
nineFive_random_df = pd.read_csv('dataset/95k-random.csv')
nineFive_random_df.head(5)

,id,account_age,no_follower,no_following,no_userfavourites,no_lists,no_tweets,no_retweets,no_hashtag,no_usermention,no_urls,no_char,no_digits,class
0,1,1235,12,31,0,0,108,0,1,0,1,30,0,spammer
1,2,695,126,569,16,0,5048,1,1,2,1,109,1,spammer
2,3,448,4,63,0,1,860,0,0,0,1,36,4,spammer
3,4,1322,8,294,2,0,237,0,0,0,1,35,2,spammer
4,5,111,1461,1365,438,10,6894,0,1,0,1,55,6,spammer


In [27]:
nineFive_random_df = nineFive_random_df.drop('id', axis=1)

# Data Pre-Processing

In [28]:
nineFive_random_df['class'].value_counts()

non-spammer    95000
spammer         5000
Name: class, dtype: int64

In [29]:
X = nineFive_random_df.drop('class', axis=1)
y = nineFive_random_df['class']
print(X.shape, y.shape)

(100000, 12) (100000,)


In [30]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [31]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [32]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [33]:
# Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create a Deep Learning Model

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [35]:
# Create model, add layers & hidden nodes
model = Sequential()
model.add(Dense(units=36, activation='relu', input_dim=12))
model.add(Dense(units=36, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [36]:
# Compile and fit the model
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 36)                468       
                                                                 
 dense_4 (Dense)             (None, 36)                1332      
                                                                 
 dense_5 (Dense)             (None, 2)                 74        
                                                                 
Total params: 1,874
Trainable params: 1,874
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=20,
    shuffle=True,
    verbose=2
)

Epoch 1/20
2344/2344 - 3s - loss: 0.1905 - accuracy: 0.9495 - 3s/epoch - 1ms/step
Epoch 2/20
2344/2344 - 2s - loss: 0.1704 - accuracy: 0.9509 - 2s/epoch - 944us/step
Epoch 3/20
2344/2344 - 3s - loss: 0.1648 - accuracy: 0.9516 - 3s/epoch - 1ms/step
Epoch 4/20
2344/2344 - 3s - loss: 0.1610 - accuracy: 0.9531 - 3s/epoch - 1ms/step
Epoch 5/20
2344/2344 - 2s - loss: 0.1585 - accuracy: 0.9539 - 2s/epoch - 966us/step
Epoch 6/20
2344/2344 - 2s - loss: 0.1559 - accuracy: 0.9543 - 2s/epoch - 995us/step
Epoch 7/20
2344/2344 - 2s - loss: 0.1541 - accuracy: 0.9550 - 2s/epoch - 959us/step
Epoch 8/20
2344/2344 - 2s - loss: 0.1526 - accuracy: 0.9554 - 2s/epoch - 966us/step
Epoch 9/20
2344/2344 - 2s - loss: 0.1514 - accuracy: 0.9555 - 2s/epoch - 1ms/step
Epoch 10/20
2344/2344 - 2s - loss: 0.1502 - accuracy: 0.9563 - 2s/epoch - 1ms/step
Epoch 11/20
2344/2344 - 2s - loss: 0.1491 - accuracy: 0.9565 - 2s/epoch - 1ms/step
Epoch 12/20
2344/2344 - 2s - loss: 0.1478 - accuracy: 0.9565 - 2s/epoch - 991us/step
E

In [39]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Loss: {model_loss}, Accuracy: {model_accuracy}")

782/782 - 1s - loss: 0.1407 - accuracy: 0.9588 - 737ms/epoch - 943us/step
Loss: 0.1406787633895874, Accuracy: 0.958840012550354


In [40]:
encoded_predictions = model.predict_classes(X_test_scaled[:100])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [41]:
predict = model.predict(X_test_scaled[:100])
labels = np.argmax(predict, axis=1)
prediction_labels = label_encoder.inverse_transform(labels)

4/4 [==============================] - 0s 1ms/step


In [42]:
print(f"Predicted Classes: {prediction_labels}")
print(f"Actual labels: {list(y_test[:100])}")

Predicted Classes: ['non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer'
 'non-spammer' 'non-spammer' 'non-spammer' 'non-spammer' 'non-

In [46]:
import joblib
joblib.dump(scaler,"bot_scaler.pkl")

NameError: name 'scaler' is not defined

In [43]:
model.save("bot_trained.h5")